#### Context
I was interested in analyzing how NFL offenses have evolved over the modern era. I used www.pro-football-reference.com to find the data that I was interested in analyzing. I used Python BeautifulSoup4 and a selenium Webdriver to pull the html data from the website and load it into a DataFrame for analysis.

##### Content
There is currently data on 672 Teams across 21 seasons. The stats included are regarding team record, passing, rushing, penalties, turnovers, and more.
The file team_stats_2003_2023.csv contains 35 variables outlined below.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

team_stats = pd.read_csv("team_stats_2003_2023.csv")
team_stats

,year,team,wins,losses,win_loss_perc,points,points_opp,points_diff,mov,g,...,rush_td,rush_yds_per_att,rush_fd,penalties,penalties_yds,pen_fd,score_pct,turnover_pct,exp_pts_tot,ties
0,2003,New England Patriots,14,2,0.875,348,238,110,6.9,16,...,9,3.4,91,111,998,26,27.9,11.3,-136.51,NaN
1,2003,Miami Dolphins,10,6,0.625,311,261,50,3.1,16,...,14,3.7,99,103,913,22,28.1,17.2,-177.92,NaN
2,2003,Buffalo Bills,6,10,0.375,243,279,-36,-2.3,16,...,13,3.9,96,106,891,22,21.9,17.6,-230.07,NaN
3,2003,New York Jets,6,10,0.375,283,299,-16,-1.0,16,...,8,4.0,78,69,550,15,32.4,11.8,-107.89,NaN
4,2003,Baltimore Ravens,10,6,0.625,391,281,110,6.9,16,...,18,4.8,115,126,970,23,31.8,16.6,-220.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,2023,Carolina Panthers,2,15,0.118,236,416,-180,-10.6,17,...,7,4.0,111,103,845,36,25.0,11.1,-135.03,NaN
668,2023,San Francisco 49ers,12,5,0.706,491,298,193,11.4,17,...,27,4.8,147,101,933,29,45.3,10.1,231.48,NaN
669,2023,Los Angeles Rams,10,7,0.588,404,377,27,1.6,17,...,18,4.3,110,89,720,35,41.1,9.2,133.32,NaN
670,2023,Seattle Seahawks,9,8,0.529,364,402,-38,-2.2,17,...,11,4.1,93,111,954,35,38.8,8.4,62.33,NaN


##### note that a lot of EDA needs to be done on the data

In [7]:
team_stats.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  672 non-null    int64  
 1   team                  672 non-null    object 
 2   wins                  672 non-null    int64  
 3   losses                672 non-null    int64  
 4   win_loss_perc         672 non-null    float64
 5   points                672 non-null    int64  
 6   points_opp            672 non-null    int64  
 7   points_diff           672 non-null    int64  
 8   mov                   352 non-null    float64
 9   g                     672 non-null    int64  
 10  total_yards           672 non-null    int64  
 11  plays_offense         672 non-null    int64  
 12  yds_per_play_offense  672 non-null    float64
 13  turnovers             672 non-null    int64  
 14  fumbles_lost          672 non-null    int64  
 15  first_down            6

In [9]:
team_stats = team_stats.drop(['ties', 'mov'], axis=1)
team_stats.columns

Index(['year', 'team', 'wins', 'losses', 'win_loss_perc', 'points',
       'points_opp', 'points_diff', 'g', 'total_yards', 'plays_offense',
       'yds_per_play_offense', 'turnovers', 'fumbles_lost', 'first_down',
       'pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_int',
       'pass_net_yds_per_att', 'pass_fd', 'rush_att', 'rush_yds', 'rush_td',
       'rush_yds_per_att', 'rush_fd', 'penalties', 'penalties_yds', 'pen_fd',
       'score_pct', 'turnover_pct', 'exp_pts_tot'],
      dtype='object')

In [10]:
team_stats.isna().sum()

year                    0
team                    0
wins                    0
losses                  0
win_loss_perc           0
points                  0
points_opp              0
points_diff             0
g                       0
total_yards             0
plays_offense           0
yds_per_play_offense    0
turnovers               0
fumbles_lost            0
first_down              0
pass_cmp                0
pass_att                0
pass_yds                0
pass_td                 0
pass_int                0
pass_net_yds_per_att    0
pass_fd                 0
rush_att                0
rush_yds                0
rush_td                 0
rush_yds_per_att        0
rush_fd                 0
penalties               0
penalties_yds           0
pen_fd                  0
score_pct               0
turnover_pct            0
exp_pts_tot             0
dtype: int64

In [11]:
team_list = team_stats['team'].unique()
no_teams = len(team_list)

print(f"there are {no_teams} teams in the dataset")

there are 37 teams in the dataset


In [12]:
# named the wlp(wins, losses, points to save strength)

wlp = team_stats.groupby(by=['team'])[['wins', 'losses', 'points']].sum()
wlp['total_games'] = wlp['wins'] + wlp['losses']
wlp.sort_values(by='points', ascending=False)


,wins,losses,points,total_games
team,,,,
New England Patriots,241,98,9083,339
New Orleans Saints,194,145,8760,339
Green Bay Packers,207,130,8648,337
Indianapolis Colts,204,134,8350,338
Philadelphia Eagles,194,143,8321,337
Dallas Cowboys,195,144,8279,339
Kansas City Chiefs,196,143,8129,339
Seattle Seahawks,197,141,7984,338
Baltimore Ravens,203,136,7913,339


In [ ]:
# rectify the issue with this function: reference to the matplotlib library

def plot(df, col1, *col2, xlabel, ylabel, title):
    plt.bar(height=df.index, x=df[col1],)
    plt.title(title)
    plt.xlabel(xlabel=xlabel)
    plt.ylabel(ylabel=ylabel)
    plt.show()
plot(df=wlp, col1='wins', xlabel='teams', ylabel='wins', title='wins per team')

#### Pobably use machine learning to create a model that can predict team wins 

In [ ]:
# this is going to involve supervised learning so the sklearn module is going to be used

from lazypredict.Supervised import LazyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
# from sklearn.
# incase i need to make an ensemble of models
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix


In [ ]:
x = []
y = []

shuffle_seed = 200

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3,
                                                 random_state=shuffle_seed,
                                                   shuffle=True)

In [ ]:
# using lazypredict module to see the best regression models for running the predictions 

lazypred = LazyRegressor(regressors='all', ignore_warnings=True)
models, predictions = lazypred.fit()
models

In [ ]:
decison = DecisionTreeRegressor(criterion=['squared_error', 'absolute_error'])
linear = LinearRegression()

voting = VotingRegressor(estimators=[('lr', linear), ('rf', forest), ('dt', decison)])
voting.fit()
voting_pred = voting.predict()
print(f"score for voting is {accuracy_score(ytest, voting_pred)}")

forest = RandomForestRegressor(n_estimators=400,
                               criterion=['squared_error', 'absolute_error'],
                               min_samples_leaf=0.12)
forest.fit()
forest_pred = forest.predict()
print(f"score of forest is {accuracy_score(ytest, forest_pred)}")

In [ ]:
# using keras sequiential model for regression purpose, will compare between this and the sklearn models

from keras import estimator, utils
import keras
import tensorflow as tf

